In [1]:
# %% Import Libraries and Data

# Third-party imports
import numpy as np
import pyansys
from pyOpt import Optimization
from pyOpt import ALPSO
import time
from mpi4py import MPI
import pandas as pd
import os
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual
import matplotlib.pyplot as plt

# Local imports
from util_mapdl import Material
from femodel import Threepartmodel

In [2]:
# %% Run ANSYS and instantiate FE-Model

ansys_path = '/home/y0065120/Dokumente/Leichtwerk/Projects/ansys-0/'
jobname ='job-0'

mapdl = pyansys.launch_mapdl(run_location=ansys_path,
                             nproc=2,
                             override=True,
                             loglevel='error',
                             additional_switches='-smp -d X11C',
                             jobname=jobname,
                             allow_ignore=True,
                             mode='console')

n_sec = 20

femodel = Threepartmodel(mapdl,
                         mesh_density_factor=1,
                         propeller = [],
                         n_sec=n_sec)

femodel.materials = {'flaxpreg': Material(mapdl, 'FLAXPREG-T-UD', 1),
                     'balsa': Material(mapdl, 'balsaholz', 2)}

femodel.element_data = pd.read_csv('./mf3218/element_data.csv', index_col=(0))

In [3]:
items = {}
items['phi0'] =widgets.FloatSlider(value=90.,min=0.,max=180., description='phi0')
items['phi1'] =widgets.FloatSlider(value=90.,min=0.,max=180., description='phi1')

for i in range(20):
    items['rho' + str(i)] = widgets.FloatSlider(value=0.5,min=0.,max=1.,step=0.01, description='rho' + str(i))
    items['div' + str(i)] = widgets.FloatSlider(value=0.5,min=0.,max=1.,step=0.01, description='div' + str(i))

In [4]:
i_ref = 1.

def pnorm(x, p):
    return (np.abs(x)**p).sum() ** (1./p)

In [5]:
@interact_manual(**items)
def objfunc(**kwargs):
    x = []
    x.append([])
    x.append(kwargs['phi0'])
    x.append(kwargs['phi1'])
    
    for i in range(20):
        x.append(kwargs['rho' + str(i)])
        x.append(kwargs['div' + str(i)])
        
    mass, i, h = femodel.evaluate(x)
    g = list(np.array(i) - i_ref)
    f = pnorm(np.array(g[:20]), 2.5)
    
    fig, ax = plt.subplots(figsize=(25.5, 15))
    
    plt.subplot(4,1,1)
    
    plt.plot(g[:20], marker='x')
    plt.grid(True,which= 'major',linestyle= '--')
    plt.xlim(0,19)
    plt.ylim(-1,0.5)
    plt.xlabel('Section')
    plt.ylabel('I_f')
    plt.xticks(np.linspace(0,19,20))
    
    plt.subplot(4,1,2)
    
    plt.plot(g[20:], marker='x')
    plt.grid(True,which= 'major',linestyle= '--')
    plt.xlim(0,19)
    plt.ylim(-1,0.5)
    plt.xlabel('Section')
    plt.ylabel('I_m')
    plt.xticks(np.linspace(0,19,20))
    
    plt.subplot(4,1,3)
    
    plt.plot(x[3::2], marker='x')
    plt.grid(True,which= 'major',linestyle= '--')
    plt.xlim(0,19)
    plt.ylim(0,1)
    plt.xlabel('Section')
    plt.ylabel('rho')
    plt.xticks(np.linspace(0,19,20))
    
    plt.subplot(4,1,4)
    
    plt.plot(x[4::2], marker='x')
    plt.grid(True,which= 'major',linestyle= '--')
    plt.xlim(0,19)
    plt.ylim(0,1)
    plt.xlabel('Section')
    plt.ylabel('div')
    plt.xticks(np.linspace(0,19,20))
    
    print(x)
    print(f)
    print(max(g))
    print(mass)

interactive(children=(FloatSlider(value=90.0, description='phi0', max=180.0), FloatSlider(value=90.0, descript…

In [6]:
x=[90.0, 90.0, 0.32, 0.5, 0.15, 0.5, 0.23, 0.5, 0.35, 0.5, 0.35, 0.5, 0.35, 0.5, 0.31, 0.5, 0.28, 0.5, 0.24, 0.5, 0.2, 0.5, 0.15, 0.5, 0.12, 0.5, 0.08, 0.5, 0.02, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5]
global_vars=x[:2]
rho=x[2::2]
div=x[3::2]
args=[]
for i in range(20):
    args.append((rho[i], div[i]))


In [ ]:
femodel.clear()
femodel.cdread()
femodel.change_design_variables(global_vars, *args)
femodel.__solve__()
f,g,h=femodel.post_processing()

max(g)
#max(h)

mapdl.open_gui()

In [8]:
elements = femodel.element_data[femodel.element_data['Section Number'] == 3]['Element Number']
len(elements)

45

In [9]:
femodel.clear()